# Prediction with Holdout Set

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import json
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# read csv file
df = pd.read_csv('data/kc_house_data_test_features.csv', index_col=0)

In [ ]:
# data preprocessing
df['sale_date'] = [x[:8] for x in df.date]
df.sale_date = df.sale_date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))
df.drop(columns='date', inplace=True)
df.drop(['id'], inplace=True, axis=1)
df.replace({'bedrooms': {33: 3}}, inplace=True)
df.replace({'bedrooms': {11: 1}}, inplace=True)
df.replace({'bathrooms': {0: 0.25}}, inplace=True)
df['sale_age'] = df.sale_date.dt.year - df[['yr_built', 'yr_renovated']].max(axis=1)
df.replace({'sale_age': {-1: 0}}, inplace=True)
df['age'] = df.sale_date.dt.year - df.yr_built
df.replace({'age': {-1: 0}}, inplace=True)
df['renovated'] = df.yr_renovated.apply(lambda x: x if x==0 else 1)
df['basement'] = df.sqft_basement.apply(lambda x: x if x==0 else 1)
df['viewed'] = df.view.apply(lambda x: x if x==0 else 1)
df.drop(['yr_built', 'yr_renovated', 'sale_date', 'sqft_basement', 'view'], inplace=True, axis=1)

In [ ]:
# dummy variables
index_dum = df[['bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'zipcode']].columns
df_dum = pd.get_dummies(data=df, columns=index_dum, drop_first=True, prefix=['bdr', 'bth', 'flr', 'cnd', 'grd', 'zip'])
# polynomial and interaction features
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_data = poly.fit_transform(df_dum)
poly_columns = poly.get_feature_names(df_dum.columns)
df_poly = pd.DataFrame(poly_data, columns=poly_columns)

In [ ]:
# subset identified by K-Best
features = df_poly[['sqft_living', 'sqft_above', 'sqft_living15', 'sqft_living^2',
       'sqft_living sqft_above', 'sqft_living zipcode', 'sqft_living lat',
       'sqft_living long', 'sqft_living sqft_living15', 'sqft_living viewed',
       'sqft_above^2', 'sqft_above zipcode', 'sqft_above lat',
       'sqft_above long', 'sqft_above sqft_living15', 'sqft_above viewed',
       'zipcode sqft_living15', 'lat sqft_living15', 'long sqft_living15',
       'sqft_living15^2']]

In [ ]:
# Scaling
scaler = StandardScaler()
features = pd.DataFrame(data=scaler.fit_transform(features), columns=features.columns)

In [ ]:
# Load pickle
with open('data/model.pickle', 'rb') as file:
    final_answer = pickle.load(file)
final_answers = final_answer.predict(features)

In [ ]:
# Write prediction to CSV file
pd.DataFrame(final_answers, columns=['predictions']).to_csv('housing_preds_Steven_Yan.csv')
